In [2]:
import numpy as np 
import matplotlib.pyplot as plt
from qutip import Qobj, basis
from qutip import rand_ket, rand_herm
from qutip import mesolve, sigmax, sigmay, sigmaz, destroy, tensor
import qutip

def dephasing_lindbladian(dim):
    """
    Construct the dephasing lindbladian superoperator.
    """
    # Initialize the lindbladian superoperator as a zero matrix
    L = Qobj(np.zeros((dim * dim, dim * dim)), dims=[[dim, dim], [dim, dim]])

    # Iterate over the dimension of the system
    for i in range(dim):
        # Add the dephasing term to the lindbladian superoperator
        L += np.sqrt(0.05) * (basis(dim, i) * basis(dim, i).dag() - 0.5 * Qobj(np.eye(dim), dims=[[dim, dim], [dim, dim]])) 

    # Return the constructed lindbladian superoperator
    return L

def wigner_fock_distribution(psi, ax):
    """
    Calculate Wigner function and plot it.
    """
    # Calculate Wigner function
    wigner = qutip.wigner(psi, xvec=np.linspace(-10,10,200), yvec=np.linspace(-10,10,200))
    
    # Plot Wigner function as contour plot
    ax.contourf(wigner.T, 50, cmap='RdBu')
    
    # Set axis labels and title
    ax.set_xlabel('$\\Re(\\langle x \\rangle)$')
    ax.set_ylabel('$\\Im(\\langle x \\rangle)$')
    ax.set_title('Wigner function of the state')
    

class Observer:
    def __init__(self, dim):
        self.dim = dim
        self.H = Qobj(rand_herm(dim))  
        self.psi = Qobj(rand_ket(dim))

    def evolve(self, dt, N):
        obs_decay = [np.sqrt(0.1) * destroy(self.dim)]
        obs_noise = [Qobj(dephasing_lindbladian(self.dim))]
        psi_t = mesolve(self.H, self.psi, np.linspace(0,dt,N), obs_decay, obs_noise)
        self.psi = psi_t[-1]
        
    def measure(self, system):
        system.psi = tensor(self.psi, system.psi).ptrace(0)

    def visualize(self, ax):
        wigner_fock_distribution(self.psi, ax)

class System:

    def __init__(self, dim):
        self.dim = dim
        self.H = Qobj(rand_herm(dim))  
        self.psi = Qobj(rand_ket(dim))

    def evolve(self, dt, N):
        sys_decay = [np.sqrt(0.05) * destroy(self.dim)]
        sys_noise = [Qobj(dephasing_lindbladian(self.dim))]
        psi_t = mesolve(self.H, self.psi, np.linspace(0,dt,N), sys_decay, sys_noise)
        self.psi = psi_t[-1]

    def collapse_animate(self, eigenstates):
        for n in range(10):
            outcome = np.random.choice(len(eigenstates), p=np.abs(eigenstates)**2)
            self.psi = eigenstates[outcome] 
            self.visualize()
            plt.pause(0.2)
            
    def visualize(self, ax):
        qutip.wigner

# Set up simulation 

obs = Observer(2)
sys = System(2)

obs_H = sigmax()
sys_H = sigmay()

# Evolve and measure

obs.evolve(5, 100) 
sys.evolve(5, 100)

obs.measure(sys)

# Animate collapse

evalues, estates = sys.psi.eigenstates()
sys.collapse_animate(estates)

TypeError: Incompatible quantum object dimensions